In [86]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
import pickle    
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
import re
from collections import Counter
import preprocessor as p

#from wordcloud import WordCloud
#import matplotlib.pyplot as plt

In [123]:
loadPath = "/Users/ipinni/Library/CloudStorage/OneDrive-UniversityofLeeds/UKRI_Tweet_Data/completed/"
loadPath2 = "/Users/ipinni/Library/CloudStorage/OneDrive-UniversityofLeeds/UKRI_Tweet_Data/tweets/COP/"

In [143]:
def get_data(version):

    with open(loadPath + version + "/" + version + "topics.list" ,'rb') as config_list_file:   
        topics = pickle.load(config_list_file)

    with open(loadPath + version + "/" + version + "docs.list", 'rb') as docs_list_file:   
        docs = pickle.load(docs_list_file)

    results = pd.DataFrame({"text": docs, "topic": topics})

    tweets = pd.read_csv(loadPath + version + "/" + version + "CleanTweets.csv")

    users = pd.read_csv(loadPath2 + "tweets" + version + ".csv")
    users = users[["sourcetweet_id", "user_username"]]

    return(results, tweets, users)

In [144]:
COP20results, COP20tweets, COP20users = get_data(version = "COP20")

In [ ]:
p.set_options(p.OPT.URL)
def merge_dfs1(users, results):
  clean = []
  for i in range(len(users)):
    clean.append(p.clean(COP20tweets.text.iloc[i]))
  
  tweets["text"] = clean

  merged1 = tweets.merge(results, on="text", how = "left")
  #merged2 = merged1.merge(users, on = "tweet_id", how = "left")

  return(merged1)

In [150]:
p.set_options(p.OPT.URL)
def merge_dfs(tweets, results, users):
  clean = []
  for i in range(len(tweets)):
    clean.append(p.clean(COP20tweets.text.iloc[i]))
  
  tweets["text"] = clean

  merged1 = tweets.merge(results, on="text", how = "left")
  merged2 = merged1.merge(users, left_on = "tweet_id", right_on = "sourcetweet_id", how = "left")

  return(merged2)

In [151]:
COP20merged = merge_dfs(COP20tweets, COP20results, COP20users)

In [152]:
COP20merged

,Unnamed: 0,tweet_id,text,like_count,retweet_count,topic,sourcetweet_id,user_username
0,3,539218799464296448,"""@AlertNet: Climate change adaptation comes of...",0.0,0.0,36.0,NaN,NaN
1,11,539218387633586176,I volunteer! // Fo fo I speak english #COP20 #...,0.0,0.0,1.0,NaN,NaN
2,16,539218160281726976,.@A_Siegel @MichaelEMann @CoralMDavenport Show...,1.0,3.0,45.0,5.392182e+17,BevForte
3,16,539218160281726976,.@A_Siegel @MichaelEMann @CoralMDavenport Show...,1.0,3.0,45.0,5.392182e+17,GrnConservatism
4,16,539218160281726976,.@A_Siegel @MichaelEMann @CoralMDavenport Show...,1.0,3.0,45.0,5.392182e+17,DclareDiane
...,...,...,...,...,...,...,...,...
16744500,223473,543189399710990338,This morning we doorstepped #Gove to read #Hun...,3.0,4.0,0.0,NaN,NaN
16744501,223534,543188980544839680,Carl Sagan said it best: “Preserve and cherish...,2.0,6.0,0.0,5.431890e+17,MiriamSwaffer
16744502,223534,543188980544839680,Carl Sagan said it best: “Preserve and cherish...,2.0,6.0,0.0,5.431890e+17,TheresaTINI
16744503,223534,543188980544839680,Carl Sagan said it best: “Preserve and cherish...,2.0,6.0,0.0,5.431890e+17,NathydaMata


In [8]:
#create dictionary of all documents in each topic
topic_docs = {topic: [] for topic in set(COP20topics)}
for topic, doc in zip(COP20topics, COP20docs):
    topic_docs[topic].append(doc)

topic_docs.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, -1])

In [140]:
len(COP20users)

223535

In [15]:
COP20topic1 = topic_docs[1]

In [54]:
pd.DataFrame(topic_docs.items())

,0,1
0,0,[.@SustainUSAgents @ClimateSelfie Love the ide...
1,1,[I volunteer! // Fo fo I speak english #COP20 ...
2,2,"[""Spokesman for IPCC"" speaks!: RT @LimaCop20: ..."
3,3,[It won't cost the earth to save the planet. R...
4,4,[ICC calls on all countries to demonstrate lea...
5,5,[This week in Lima: Essential that governments...
6,6,[@LibDemPress @nick_clegg @trevdick Why not in...
7,7,[getting keen for some late nights following #...
8,8,[The beginning of the end of the utility secto...
9,9,[Why include #gender in climate #negotations? ...


In [ ]:
for x, y in topic_docs.items():
  print(x, y)

In [43]:
all = pd.DataFrame()

In [67]:
all = pd.DataFrame(columns = ["text", "topic"])
for i in set(topic_docs):
    #print(i)
    test = pd.DataFrame(topic_docs[i], columns=["text"])
    test['topic'] = i
    all.append(test)

In [69]:
for i in set(topic_docs):
    #print(i)
    test = [topic_docs[i]]

In [71]:
results = pd.DataFrame({"Doc": COP20docs, "Topic": COP20topics})

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
g = nx.Graph()
user = []
topic = []

with open("Trimester Bipartite/SepDec17.csv", "r") as f:
    f.readline()
    for l in f:
        l = l.rstrip().split(",")
        u = str(l[0])
        t = str(l[3])
        g.add_edge(u, t)
        user.append(u)
        topic.append(t)


Ngraph_user = bipartite.weighted_projected_graph(g, user)
nx.write_edgelist(Ngraph_user, "Proj_SepDec17.csv")